# Лабораторна робота № 6

##  ІП-14 Бабіч Денис

Розробити програмний скрипт, що реалізує стеження за об’єктом у цифровому відеопотоці. Зміст відео, об’єкт стеження – обрати самостійно. Метод та технологію стеження обрати такою, що забезпечує стійкість процесу object-tracking для обраних вихідними даними (відео, об’єкт стеження). Вибір обґрунтувати та довести його ефективність.

---

# Підготовчий етап

## Імпортування необхідних модулів

In [1]:
import cv2
import numpy as np

---

# Основний етап

## Застосування на прикладі знаходження рис обличчя

In [2]:
KEY_ESCAPE = 27

CLASSIFIER_EYE = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_eye.xml")
CLASSIFIER_SMILE = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_smile.xml")
CLASSIFIER_FACE = cv2.CascadeClassifier(f"{cv2.data.haarcascades}haarcascade_frontalface_default.xml")

BOUNDING_THICKNESS = 2
COLOR_EYE = (255, 0, 0)
COLOR_FACE = (0, 255, 0)
COLOR_SMILE = (0, 0, 255)

CAPTURE = cv2.VideoCapture("video1.mp4")
FPS = int(CAPTURE.get(cv2.CAP_PROP_FPS))

while True:
    ret, frame = CAPTURE.read()

    if not ret:
        break

    gray_filter = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = CLASSIFIER_FACE.detectMultiScale(gray_filter, scaleFactor = 1.05, minNeighbors = 5)

    for (x, y, width, height) in faces:
        cv2.rectangle(frame, (x, y), (x + width, y + height), COLOR_FACE, BOUNDING_THICKNESS)

        roi_color = frame[y:(y + height), x:(x + width)]
        roi_gray = gray_filter[y:(y + height), x:(x + width)]

        eyes = CLASSIFIER_EYE.detectMultiScale(roi_gray, scaleFactor = 1.1, minNeighbors = 5)

        for (x, y, width, height) in eyes:
            cv2.rectangle(roi_color, (x, y), (x + width, y + height), COLOR_EYE, BOUNDING_THICKNESS)

        smiles = CLASSIFIER_SMILE.detectMultiScale(roi_gray, scaleFactor = 1.2, minNeighbors = 125)

        for (x, y, width, height) in smiles:
            cv2.rectangle(roi_color, (x, y), (x + width, y + height), COLOR_SMILE, BOUNDING_THICKNESS)

    cv2.imshow("Detected face features", frame)

    if cv2.waitKey(FPS) & 0xFF == KEY_ESCAPE:
        break

CAPTURE.release()
cv2.destroyAllWindows()

## Застосування на прикладі знаходження людей на відео

In [3]:
KEY_ESCAPE = 27

HOG_DESCRIPTOR = cv2.HOGDescriptor()
HOG_DESCRIPTOR.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

BOUNDING_THICKNESS = 2
COLOR_HUMAN = (0, 255, 255)

CAPTURE = cv2.VideoCapture("video2.mp4")
FPS = int(CAPTURE.get(cv2.CAP_PROP_FPS))

while True:
    _, frame = CAPTURE.read()

    if not ret:
        break

    gray_filter = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    people = HOG_DESCRIPTOR.detectMultiScale(gray_filter, winStride = (8, 8), padding = (16, 16), scale = 1.25)
    people = np.array([[human_x, human_y, human_x + human_w, human_y + human_h] for (human_x, human_y, human_w, human_h) in people[0]])

    for (x, y, width, height) in people:
        cv2.rectangle(frame, (x, y), (width, height), COLOR_HUMAN, BOUNDING_THICKNESS)

    cv2.imshow("Detected individuals", frame)

    if cv2.waitKey(FPS) & 0xFF == KEY_ESCAPE:
        break

CAPTURE.release()
cv2.destroyAllWindows()